In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Regression in Python<br>
<br>
***<br>
This is a very quick run-through of some basic statistical concepts, adapted from [Lab 4 in Harvard's CS109](https://github.com/cs109/2015lab4) course. Please feel free to try the original lab if you're feeling ambitious :-) The CS109 git repository also has the solutions if you're stuck.<br>
<br>
* Linear Regression Models<br>
* Prediction using linear regression<br>
<br>
Linear regression is used to model and predict continuous outcomes with normal random errors. There are nearly an infinite number of different types of regression models and each regression model is typically defined by the distribution of the prediction errors (called "residuals") of the type of data. Logistic regression is used to model binary outcomes whereas Poisson regression is used to predict counts. In this exercise, we'll see some examples of linear regression as well as Train-test splits.<br>
<br>
The packages we'll cover are: `statsmodels`, `seaborn`, and `scikit-learn`. While we don't explicitly teach `statsmodels` and `seaborn` in the Springboard workshop, those are great libraries to know.<br>
***

<img width=600 height=300 src="https://imgs.xkcd.com/comics/sustainable.png"/><br>
***

In[1]:

special IPython command to prepare the notebook for matplotlib and other libraries

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn

In [ ]:
import seaborn as sns

special matplotlib argument for improved plots

In [ ]:
from matplotlib import rcParams
sns.set_style("whitegrid")
sns.set_context("poster")

***<br>
# Part 1: Introduction to Linear Regression<br>
### Purpose of linear regression<br>
***<br>
<div class="span5 alert alert-info"><br>
<br>
<p> Given a dataset containing predictor variables $X$ and outcome/response variable $Y$, linear regression can be used to: </p><br>
<ul><br>
  <li> Build a <b>predictive model</b> to predict future values of $\hat{Y}$, using new data $X^*$ where $Y$ is unknown.</li><br>
  <li> Model the <b>strength of the relationship</b> between each independent variable $X_i$ and $Y$</li><br>
    <ul><br>
      <li> Many times, only a subset of independent variables $X_i$ will have a linear relationship with $Y$</li><br>
      <li> Need to figure out which $X_i$ contributes most information to predict $Y$ </li><br>
    </ul><br>
   <li>It is in many cases, the first pass prediction algorithm for continuous outcomes. </li><br>
</ul><br>
</div><br>
<br>
### A Brief Mathematical Recap<br>
***<br>
<br>
[Linear Regression](http://en.wikipedia.org/wiki/Linear_regression) is a method to model the relationship between a set of independent variables $X$ (also knowns as explanatory variables, features, predictors) and a dependent variable $Y$.  This method assumes the relationship between each predictor $X$ is **linearly** related to the dependent variable $Y$. The most basic linear regression model contains one independent variable $X$, we'll call this the simple model. <br>
<br>
$$ Y = \beta_0 + \beta_1 X + \epsilon$$<br>
<br>
where $\epsilon$ is considered as an unobservable random variable that adds noise to the linear relationship. In linear regression, $\epsilon$ is assumed to be normally distributed with a mean of 0. In other words, what this means is that on average, if we know $Y$, a roughly equal number of predictions $\hat{Y}$ will be above $Y$ and others will be below $Y$. That is, on average, the error is zero. The residuals, $\epsilon$ are also assumed to be "i.i.d.": independently and identically distributed. Independence means that the residuals are not correlated -- the residual from one prediction has no effect on the residual from another prediction. Correlated errors are common in time series analysis and spatial analyses.<br>
<br>
* $\beta_0$ is the intercept of the linear model and represents the average of $Y$ when all independent variables $X$ are set to 0.<br>
<br>
* $\beta_1$ is the slope of the line associated with the regression model and represents the average effect of a one-unit increase in $X$ on $Y$.<br>
<br>
* Back to the simple model. The model in linear regression is the *conditional mean* of $Y$ given the values in $X$ is expressed a linear function.  <br>
<br>
$$ y = f(x) = E(Y | X = x)$$ <br>
<br>
![conditional mean](images/conditionalmean.png)<br>
http://www.learner.org/courses/againstallodds/about/glossary.html<br>
<br>
* The goal is to estimate the coefficients (e.g. $\beta_0$ and $\beta_1$). We represent the estimates of the coefficients with a "hat" on top of the letter.  <br>
<br>
$$ \hat{\beta}_0, \hat{\beta}_1 $$<br>
<br>
* Once we estimate the coefficients $\hat{\beta}_0$ and $\hat{\beta}_1$, we can use these to predict new values of $Y$ given new data $X$.<br>
<br>
$$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x_1$$<br>
<br>
* Multiple linear regression is when you have more than one independent variable and the estimation involves matrices<br>
    * $X_1$, $X_2$, $X_3$, $\ldots$<br>
<br>
<br>
* How do you estimate the coefficients? <br>
    * There are many ways to fit a linear regression model<br>
    * The method called **least squares** is the most common methods<br>
    * We will discuss least squares<br>
<br>
$$ Y = \beta_0 + \beta_1 X_1 + \ldots + \beta_p X_p + \epsilon$$ <br>
    <br>
### Estimating $\hat\beta$: Least squares<br>
***<br>
[Least squares](http://en.wikipedia.org/wiki/Least_squares) is a method that can estimate the coefficients of a linear model by minimizing the squared residuals: <br>
<br>
$$ \mathscr{L} = \sum_{i=1}^N \epsilon_i^2 = \sum_{i=1}^N \left( y_i - \hat{y}_i \right)^2  = \sum_{i=1}^N \left(y_i - \left(\beta_0 + \beta_1 x_i\right)\right)^2 $$<br>
<br>
where $N$ is the number of observations and $\epsilon$ represents a residual or error, ACTUAL - PREDICTED.  <br>
<br>
#### Estimating the intercept $\hat{\beta_0}$ for the simple linear model<br>
<br>
We want to minimize the squared residuals and solve for $\hat{\beta_0}$ so we take the partial derivative of $\mathscr{L}$ with respect to $\hat{\beta_0}$ 

$<br>
\begin{align}<br>
\frac{\partial \mathscr{L}}{\partial \hat{\beta_0}} &= \frac{\partial}{\partial \hat{\beta_0}} \sum_{i=1}^N \epsilon^2 \\<br>
&= \frac{\partial}{\partial \hat{\beta_0}} \sum_{i=1}^N \left( y_i - \hat{y}_i \right)^2 \\<br>
&= \frac{\partial}{\partial \hat{\beta_0}} \sum_{i=1}^N \left( y_i - \left( \hat{\beta}_0 + \hat{\beta}_1 x_i \right) \right)^2 \\<br>
&= -2 \sum_{i=1}^N \left( y_i - \left( \hat{\beta}_0 + \hat{\beta}_1 x_i \right) \right) \hspace{25mm} \mbox{(by chain rule)} \\<br>
&= -2 \sum_{i=1}^N (y_i - \hat{\beta}_0 - \hat{\beta}_1 x_i) \\<br>
&= -2 \left[ \left( \sum_{i=1}^N y_i \right) - N \hat{\beta_0} - \hat{\beta}_1 \left( \sum_{i=1}^N x_i<br>
\right) \right] \\<br>
& 2 \left[ N \hat{\beta}_0 + \hat{\beta}_1 \sum_{i=1}^N x_i - \sum_{i=1}^N y_i \right] = 0 \hspace{20mm} \mbox{(Set equal to 0 and solve for $\hat{\beta}_0$)} \\<br>
& N \hat{\beta}_0 + \hat{\beta}_1 \sum_{i=1}^N x_i - \sum_{i=1}^N y_i = 0 \\<br>
& N \hat{\beta}_0 = \sum_{i=1}^N y_i - \hat{\beta}_1 \sum_{i=1}^N x_i \\<br>
& \hat{\beta}_0 = \frac{\sum_{i=1}^N y_i - \hat{\beta}_1 \sum_{i=1}^N x_i}{N} \\<br>
& \hat{\beta}_0 = \frac{\sum_{i=1}^N y_i}{N} - \hat{\beta}_1 \frac{\sum_{i=1}^N x_i}{N} \\<br>
& \boxed{\hat{\beta}_0 = \bar{y} - \hat{\beta}_1 \bar{x}}<br>
\end{align}<br>
$

Using this new information, we can compute the estimate for $\hat{\beta}_1$ by taking the partial derivative of $\mathscr{L}$ with respect to $\hat{\beta}_1$.

$<br>
\begin{align}<br>
\frac{\partial \mathscr{L}}{\partial \hat{\beta_1}} &= \frac{\partial}{\partial \hat{\beta_1}} \sum_{i=1}^N \epsilon^2 \\<br>
&= \frac{\partial}{\partial \hat{\beta_1}} \sum_{i=1}^N \left( y_i - \hat{y}_i \right)^2 \\<br>
&= \frac{\partial}{\partial \hat{\beta_1}} \sum_{i=1}^N \left( y_i - \left( \hat{\beta}_0 + \hat{\beta}_1 x_i \right) \right)^2 \\<br>
&= 2 \sum_{i=1}^N \left( y_i - \left( \hat{\beta}_0 + \hat{\beta}_1 x_i \right) \right) \left( -x_i \right) \hspace{25mm}\mbox{(by chain rule)} \\<br>
&= -2 \sum_{i=1}^N x_i \left( y_i - \hat{\beta}_0 - \hat{\beta}_1 x_i \right) \\<br>
&= -2 \sum_{i=1}^N x_i (y_i - \hat{\beta}_0 x_i - \hat{\beta}_1 x_i^2) \\<br>
&= -2 \sum_{i=1}^N x_i (y_i - \left( \bar{y} - \hat{\beta}_1 \bar{x} \right) x_i - \hat{\beta}_1 x_i^2) \\<br>
&= -2 \sum_{i=1}^N (x_i y_i - \bar{y}x_i + \hat{\beta}_1\bar{x}x_i - \hat{\beta}_1 x_i^2) \\<br>
&= -2 \left[ \sum_{i=1}^N x_i y_i - \bar{y} \sum_{i=1}^N x_i + \hat{\beta}_1\bar{x}\sum_{i=1}^N x_i - \hat{\beta}_1 \sum_{i=1}^N x_i^2 \right] \\<br>
&= -2 \left[ \hat{\beta}_1 \left\{ \bar{x} \sum_{i=1}^N x_i - \sum_{i=1}^N x_i^2 \right\} + \left\{ \sum_{i=1}^N x_i y_i - \bar{y} \sum_{i=1}^N x_i \right\}\right] \\<br>
& 2 \left[ \hat{\beta}_1 \left\{ \sum_{i=1}^N x_i^2 - \bar{x} \sum_{i=1}^N x_i \right\} + \left\{ \bar{y} \sum_{i=1}^N x_i - \sum_{i=1}^N x_i y_i \right\} \right] = 0 \\<br>
& \hat{\beta}_1 = \frac{-\left( \bar{y} \sum_{i=1}^N x_i - \sum_{i=1}^N x_i y_i \right)}{\sum_{i=1}^N x_i^2 - \bar{x}\sum_{i=1}^N x_i} \\<br>
&= \frac{\sum_{i=1}^N x_i y_i - \bar{y} \sum_{i=1}^N x_i}{\sum_{i=1}^N x_i^2 - \bar{x} \sum_{i=1}^N x_i} \\<br>
& \boxed{\hat{\beta}_1 = \frac{\sum_{i=1}^N x_i y_i - \bar{x}\bar{y}n}{\sum_{i=1}^N x_i^2 - n \bar{x}^2}}<br>
\end{align}<br>
$

The solution can be written in compact matrix notation as<br>
<br>
$$\hat\beta =  (X^T X)^{-1}X^T Y$$ <br>
<br>
We wanted to show you this in case you remember linear algebra, in order for this solution to exist we need $X^T X$ to be invertible. Of course this requires a few extra assumptions, $X$ must be full rank so that $X^T X$ is invertible, etc. Basically, $X^T X$ is full rank if all rows and columns are linearly independent. This has a loose relationship to variables and observations being independent respective. **This is important for us because this means that having redundant features in our regression models will lead to poorly fitting (and unstable) models.** We'll see an implementation of this in the extra linear regression example.

***<br>
# Part 2: Exploratory Data Analysis for Linear Relationships<br>
<br>
The [Boston Housing data set](https://archive.ics.uci.edu/ml/datasets/Housing) contains information about the housing values in suburbs of Boston.  This dataset was originally taken from the StatLib library which is maintained at Carnegie Mellon University and is now available on the UCI Machine Learning Repository. <br>
<br>
<br>
## Load the Boston Housing data set from `sklearn`<br>
***<br>
<br>
This data set is available in the [sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html#sklearn.datasets.load_boston) python module which is how we will access it today.  

In[2]:

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd

In [ ]:
boston = load_boston()

In[3]:

In [ ]:
boston.keys()

In[4]:

In [ ]:
boston.data.shape

In[5]:

Print column names

In [ ]:
print(boston.feature_names)

In[6]:

Print description of Boston housing data set

In [ ]:
print(boston.DESCR)

Now let's explore the data set itself. 

In[7]:

In [ ]:
bos = pd.DataFrame(boston.data)
bos.head()

There are no column names in the DataFrame. Let's add those. 

In[8]:

In [ ]:
bos.columns = boston.feature_names
bos.head()

Now we have a pandas DataFrame called `bos` containing all the data we want to use to predict Boston Housing prices.  Let's create a variable called `PRICE` which will contain the prices. This information is contained in the `target` data. 

In[9]:

In [ ]:
print(boston.target.shape)

In[10]:

In [ ]:
bos['PRICE'] = boston.target
bos.head()

## EDA and Summary Statistics<br>
***<br>
<br>
Let's explore this data set.  First we use `describe()` to get basic summary statistics for each of the columns. 

In[11]:

In [ ]:
bos.describe()

### Scatterplots<br>
***<br>
<br>
Let's look at some scatter plots for three variables: 'CRIM' (per capita crime rate), 'RM' (number of rooms) and 'PTRATIO' (pupil-to-teacher ratio in schools).  

In[12]:

In [ ]:
plt.scatter(bos.CRIM, bos.PRICE)
plt.xlabel("Per capita crime rate by town (CRIM)")
plt.ylabel("Housing Price")
plt.title("Relationship between CRIM and Price")

<div class="span5 alert alert-info"><br>
<h3>Part 2 Checkup Exercise Set I</h3><br>
<br>
<p><b>Exercise:</b> What kind of relationship do you see? e.g. positive, negative?  linear? non-linear? Is there anything else strange or interesting about the data? What about outliers?</p><br>
<br>
<br>
<p><b>Exercise:</b> Create scatter plots between *RM* and *PRICE*, and *PTRATIO* and *PRICE*. Label your axes appropriately using human readable labels. Tell a story about what you see.</p><br>
<br>
<p><b>Exercise:</b> What are some other numeric variables of interest? Why do you think they are interesting? Plot scatterplots with these variables and *PRICE* (house price) and tell a story about what you see.</p><br>
<br>
</div>

In[22]:

your turn: describe relationship<br>
CRIM and Price seems seems to be negatively coorelated.LSTAT has a high negative corelation with Price and RM has a high positive coorelation. <br>
lso variables like RAD and TAX are highly coorelated, so is AGE and DIX , Nox and AGE etc.<br>


In [ ]:
plt.figure(figsize=(20,10))
correlation_matrix = bos.corr().round(2)
# annot = True to print the values inside the square
sns.heatmap(data=correlation_matrix, annot=True,cmap="YlGnBu")
plt.show()

In[19]:

your turn: scatter plot between *RM* and *PRICE*

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(bos.RM,bos.PRICE)
plt.xlabel("Avg rooms per dwelling")
plt.title("Relationships between avg rooms per dwelling and price")
plt.show()

In[18]:

your turn: scatter plot between *PTRATIO* and *PRICE*

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(bos.PTRATIO,bos.PRICE)
plt.xlabel("Pupil teacher ratio by town")
plt.title("Relationships between pupil teacher ratio by town and price")
plt.show()

### Scatterplots using Seaborn<br>
***<br>
<br>
[Seaborn](https://stanford.edu/~mwaskom/software/seaborn/) is a cool Python plotting library built on top of matplotlib. It provides convenient syntax and shortcuts for many common types of plots, along with better-looking defaults.<br>
<br>
We can also use [seaborn regplot](https://stanford.edu/~mwaskom/software/seaborn/tutorial/regression.html#functions-to-draw-linear-regression-models) for the scatterplot above. This provides automatic linear regression fits (useful for data exploration later on). Here's one example below.

In[23]:

In [ ]:
sns.regplot(y="PRICE", x="RM", data=bos, fit_reg = True)

### Histograms<br>
***<br>


In[25]:

In [ ]:
plt.hist(np.log(bos.CRIM))
plt.title("CRIM")
plt.xlabel("Crime rate per capita")
plt.ylabel("Frequency")
plt.show()

<div class="span5 alert alert-info"><br>
<h3>Part 2 Checkup Exercise Set II</h3><br>
<br>
<p><b>Exercise:</b> In the above histogram, we took the logarithm of the crime rate per capita. Repeat this histogram without taking the log. What was the purpose of taking the log? What do we gain by making this transformation? What do you now notice about this variable that is not obvious without making the transformation?<br>
<br>
<p><b>Exercise:</b> Plot the histogram for *RM* and *PTRATIO* against each other, along with the two variables you picked in the previous section. We are looking for correlations in predictors here.</p><br>
</div>

In[27]:

our turn<br>
ogarithmic transformations make the distribution more normalized which helps in identifying how skewed your data was previously. 

In [ ]:
plt.hist(bos.CRIM)
plt.title("CRIM")
plt.xlabel("Crime rate per capita")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.hist(bos.RM)
plt.title("# of Rooms")
plt.xlabel("# of Rooms")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.hist(bos.PTRATIO)
plt.title("Pupil to Teacher Ratio")
plt.xlabel("Pupil to Teacher Ratio")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.hist(bos.NOX)
plt.title("NOx Concentration")
plt.xlabel("NOx Concentration (pp10m)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.hist(bos.TAX)
plt.title("Tax rate per $10,000")
plt.xlabel("Tax Rate per $10,000")
plt.ylabel("Frequency")
plt.show()

## Part 3: Linear Regression with Boston Housing Data Example<br>
***<br>
<br>
Here, <br>
<br>
$Y$ = boston housing prices (called "target" data in python, and referred to as the dependent variable or response variable)<br>
<br>
and<br>
<br>
$X$ = all the other features (or independent variables, predictors or explanatory variables)<br>
<br>
which we will use to fit a linear regression model and predict Boston housing prices. We will use the least-squares method to estimate the coefficients.  

We'll use two ways of fitting a linear regression. We recommend the first but the second is also powerful in its features.

### Fitting Linear Regression using `statsmodels`<br>
***<br>
[Statsmodels](http://statsmodels.sourceforge.net/) is a great Python library for a lot of basic and inferential statistics. It also provides basic regression functions using an R-like syntax, so it's commonly used by statisticians. While we don't cover statsmodels officially in the Data Science Intensive workshop, it's a good library to have in your toolbox. Here's a quick example of what you could do with it. The version of least-squares we will use in statsmodels is called *ordinary least-squares (OLS)*. There are many other versions of least-squares such as [partial least squares (PLS)](https://en.wikipedia.org/wiki/Partial_least_squares_regression) and [weighted least squares (WLS)](https://en.wikipedia.org/wiki/Iteratively_reweighted_least_squares).

In[28]:

Import regression modules

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In[29]:

statsmodels works nicely with pandas dataframes<br>
The thing inside the "quotes" is called a formula, a bit on that below

In [ ]:
m = ols('PRICE ~ RM',bos).fit()
print(m.summary())

#### Interpreting coefficients<br>
<br>
There is a ton of information in this output. But we'll concentrate on the coefficient table (middle table). We can interpret the `RM` coefficient (9.1021) by first noticing that the p-value (under `P>|t|`) is so small, basically zero. This means that the number of rooms, `RM`, is a statisticall significant predictor of `PRICE`. The regression coefficient for `RM` of 9.1021 means that *on average, each additional room is associated with an increase of $\$9,100$ in house price net of the other variables*. The confidence interval gives us a range of plausible values for this average change, about ($\$8,279, \$9,925$), definitely not chump change. <br>
<br>
In general, the $\hat{\beta_i}, i > 0$ can be interpreted as the following: "A one unit increase in $x_i$ is associated with, on average, a $\hat{\beta_i}$ increase/decrease in $y$ net of all other variables."<br>
<br>
On the other hand, the interpretation for the intercept, $\hat{\beta}_0$ is the average of $y$ given that all of the independent variables $x_i$ are 0.

####  `statsmodels` formulas<br>
***<br>
This formula notation will seem familiar to `R` users, but will take some getting used to for people coming from other languages or are new to statistics.<br>
<br>
The formula gives instruction for a general structure for a regression call. For `statsmodels` (`ols` or `logit`) calls you need to have a Pandas dataframe with column names that you will add to your formula. In the below example you need a pandas data frame that includes the columns named (`Outcome`, `X1`,`X2`, ...), but you don't need to build a new dataframe for every regression. Use the same dataframe with all these things in it. The structure is very simple:<br>
<br>
`Outcome ~ X1`<br>
<br>
But of course we want to to be able to handle more complex models, for example multiple regression is doone like this:<br>
<br>
`Outcome ~ X1 + X2 + X3`<br>
<br>
In general, a formula for an OLS multiple linear regression is<br>
<br>
`Y ~ X1 + X2 + ... + Xp`<br>
<br>
This is the very basic structure but it should be enough to get you through the homework. Things can get much more complex. You can force statsmodels to treat variables as categorical with the `C()` function, call numpy functions to transform data such as `np.log` for extremely-skewed data, or fit a model without an intercept by including `- 1` in the formula. For a quick run-down of further uses see the `statsmodels` [help page](http://statsmodels.sourceforge.net/devel/example_formulas.html).<br>


Let's see how our model actually fit our data. We can see below that there is a ceiling effect, we should probably look into that. Also, for large values of $Y$ we get underpredictions, most predictions are below the 45-degree gridlines. 

<div class="span5 alert alert-info"><br>
<h3>Part 3 Checkup Exercise Set I</h3><br>
<br>
<p><b>Exercise:</b> Create a scatterplot between the predicted prices, available in `m.fittedvalues` (where `m` is the fitted model) and the original prices. How does the plot look? Do you notice anything interesting or weird in the plot? Comment on what you see.</p><br>
</div>

In[34]:

your turn

In [ ]:
sns.scatterplot(x=bos.PRICE, y=m.fittedvalues)
plt.title("Original Prices vs Predicted Prices")
plt.xlabel("Original Prices")
plt.ylabel("Predicted Prices")
plt.show()
# There is a strong relationship between actual and predicted prices except for price range 50 where differences seem slightly higher.

### Fitting Linear Regression using `sklearn`<br>


In[35]:

In [ ]:
from sklearn.linear_model import LinearRegression
X = bos.drop('PRICE', axis = 1)

This creates a LinearRegression object

In [ ]:
lm = LinearRegression()
lm

#### What can you do with a LinearRegression object? <br>
***<br>
Check out the scikit-learn [docs here](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). We have listed the main functions here. Most machine learning models in scikit-learn follow this same API of fitting a model with `fit`, making predictions with `predict` and the appropriate scoring function `score` for each model.

Main functions | Description<br>
--- | --- <br>
`lm.fit()` | Fit a linear model<br>
`lm.predit()` | Predict Y using the linear model with estimated coefficients<br>
`lm.score()` | Returns the coefficient of determination (R^2). *A measure of how well observed outcomes are replicated by the model, as the proportion of total variation of outcomes explained by the model*

#### What output can you get?

In[ ]:

Look inside lm object<br>
lm.<tab>

Output | Description<br>
--- | --- <br>
`lm.coef_` | Estimated coefficients<br>
`lm.intercept_` | Estimated intercept 

### Fit a linear model<br>
***<br>
<br>
The `lm.fit()` function estimates the coefficients the linear regression using least squares. 

In[36]:

Use all 13 predictors to fit linear regression model

In [ ]:
lm.fit(X, bos.PRICE)

<div class="span5 alert alert-info"><br>
<h3>Part 3 Checkup Exercise Set II</h3><br>
<br>
<p><b>Exercise:</b> How would you change the model to not fit an intercept term? Would you recommend not having an intercept? Why or why not? For more information on why to include or exclude an intercept, look [here](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faq-what-is-regression-through-the-origin/).</p><br>
<br>
<p><b>Exercise:</b> One of the assumptions of the linear model is that the residuals must be i.i.d. (independently and identically distributed). To satisfy this, is it enough that the residuals are normally distributed? Explain your answer.</p><br>
<br>
<p><b>Exercise:</b> True or false. To use linear regression, $Y$ must be normally distributed. Explain your answer.</p><br>
</div><br>


# your turn<br>
you can change the fit_inteercept to False for assuming not to fit an intercept term. That is not advisable however as in doing so you are saying the <br>
y intercept is zero and the model is heavily biased towards the coefficient value and the purpose of reducing the mean squared error would not be satisfied.<br>
<br>
Normal distribution of residuals wouldnt prove that they are independent and identically distributed. we could plot the residuals and see if they have a uniform distribution. Residuals are also the difference between the actual and predicted variable , and their independence lies on how independent the actual variables are.<br>
<br>
Dependent value does not have to be normally distributed. However the residuals are needed to be.<br>
<br>
<br>


### Estimated intercept and coefficients<br>
<br>
Let's look at the estimated coefficients from the linear model using `1m.intercept_` and `lm.coef_`.  <br>
<br>
After we have fit our linear regression model using the least squares method, we want to see what are the estimates of our coefficients $\beta_0$, $\beta_1$, ..., $\beta_{13}$: <br>
<br>
$$ \hat{\beta}_0, \hat{\beta}_1, \ldots, \hat{\beta}_{13} $$<br>
<br>


In[37]:

In [ ]:
print('Estimated intercept coefficient: {}'.format(lm.intercept_))

In[38]:

In [ ]:
print('Number of coefficients: {}'.format(len(lm.coef_)))

In[39]:

The coefficients

In [ ]:
pd.DataFrame({'features': X.columns, 'estimatedCoefficients': lm.coef_})[['features', 'estimatedCoefficients']]

### Predict Prices <br>
<br>
We can calculate the predicted prices ($\hat{Y}_i$) using `lm.predict`. <br>
<br>
$$ \hat{Y}_i = \hat{\beta}_0 + \hat{\beta}_1 X_1 + \ldots \hat{\beta}_{13} X_{13} $$

In[40]:

first five predicted prices

In [ ]:
lm.predict(X)[0:5]

<div class="span5 alert alert-info"><br>
<h3>Part 3 Checkup Exercise Set III</h3><br>
<br>
<p><b>Exercise:</b> Histogram: Plot a histogram of all the predicted prices. Write a story about what you see. Describe the shape, center and spread of the distribution. Are there any outliers? What might be the reason for them? Should we do anything special with them?</p><br>
<br>
<p><b>Exercise:</b> Scatterplot: Let's plot the true prices compared to the predicted prices to see they disagree (we did this with `statsmodels` before).</p><br>
<br>
<p><b>Exercise:</b> We have looked at fitting a linear model in both `statsmodels` and `scikit-learn`. What are the advantages and disadvantages of each based on your exploration? Based on the information provided by both packages, what advantage does `statsmodels` provide?</p><br>
</div>

In[41]:

your turn

In [ ]:
plt.hist(lm.predict(X), bins=15)
plt.xlabel("House Price (in $1,000s)")
plt.ylabel("Count")
plt.xticks([x for x in range(0,50,5)])
plt.title("Predicted Housing Price Histogram")
print("Center: $%f      Spread (standard deviation): $%f" % (np.mean(lm.predict(X))*1000, np.std(lm.predict(X))*1000))

They are normally distributed with the center and spread values below. There seems to be prediction values for hosuing prices less than 0<br>
hat doesnt make sense.

### Evaluating the Model: Sum-of-Squares<br>
<br>
The partitioning of the sum-of-squares shows the variance in the predictions explained by the model and the variance that is attributed to error.<br>
<br>
$$TSS = ESS + RSS$$<br>
<br>
#### Residual Sum-of-Squares (aka $RSS$)<br>
<br>
The residual sum-of-squares is one of the basic ways of quantifying how much error exists in the fitted model. We will revisit this in a bit.<br>
<br>
$$ RSS = \sum_{i=1}^N r_i^2 = \sum_{i=1}^N \left(y_i - \left(\beta_0 + \beta_1 x_i\right)\right)^2 $$

In[42]:

In [ ]:
print(np.sum((bos.PRICE - lm.predict(X)) ** 2))

#### Explained Sum-of-Squares (aka $ESS$)<br>
<br>
The explained sum-of-squares measures the variance explained by the regression model.<br>
<br>
$$ESS = \sum_{i=1}^N \left( \hat{y}_i - \bar{y} \right)^2 = \sum_{i=1}^N \left( \left( \hat{\beta}_0 + \hat{\beta}_1 x_i \right) - \bar{y} \right)^2$$

In[43]:

In [ ]:
print(np.sum((lm.predict(X) - np.mean(bos.PRICE))**2))

### Evaluating the Model: The Coefficient of Determination ($R^2$)<br>
<br>
The coefficient of determination, $R^2$, tells us the percentage of the variance in the response variable $Y$ that can be explained by the linear regression model.<br>
<br>
$$ R^2 = \frac{ESS}{TSS} $$<br>
<br>
The $R^2$ value is one of the most common metrics that people use in describing the quality of a model, but it is important to note that *$R^2$ increases artificially as a side-effect of increasing the number of independent variables.* While $R^2$ is reported in almost all statistical packages, another metric called the *adjusted $R^2$* is also provided as it takes into account the number of variables in the model, and can sometimes even be used for non-linear regression models!<br>
<br>
$$R_{adj}^2 = 1 - \left( 1 - R^2 \right) \frac{N - 1}{N - K - 1} = R^2 - \left( 1 - R^2 \right) \frac{K}{N - K - 1} = 1 - \frac{\frac{RSS}{DF_R}}{\frac{TSS}{DF_T}}$$<br>
<br>
where $N$ is the number of observations, $K$ is the number of variables, $DF_R = N - K - 1$ is the degrees of freedom associated with the residual error and $DF_T = N - 1$ is the degrees of the freedom of the total error.

### Evaluating the Model: Mean Squared Error and the $F$-Statistic<br>
***<br>
The mean squared errors are just the *averages* of the sum-of-squares errors over their respective degrees of freedom.<br>
<br>
$$MSR = \frac{ESS}{K}$$<br>
<br>
$$MSE = \frac{RSS}{N-K-1}$$<br>
<br>
**Remember:** Notation may vary across resources particularly the use of *R* and *E* in *RSS/ESS* and *MSR/MSE*. In some resources, E = explained and R = residual. In other resources, E = error and R = regression (explained). **This is a very important distinction that requires looking at the formula to determine which naming scheme is being used.**<br>
<br>
Given the MSR and MSE, we can now determine whether or not the entire model we just fit is even statistically significant. We use an $F$-test for this. The null hypothesis is that all of the $\beta$ coefficients are zero, that is, none of them have any effect on $Y$. The alternative is that *at least one* $\beta$ coefficient is nonzero, but it doesn't tell us which one in a multiple regression:<br>
<br>
$$H_0: \beta_i = 0, \mbox{for all $i$} \\<br>
H_A: \beta_i > 0, \mbox{for some $i$}$$ <br>
<br>
$$F = \frac{MSR}{MSE} = \left( \frac{R^2}{1 - R^2} \right) \left( \frac{N - K - 1}{K} \right)$$<br>
 <br>
Once we compute the $F$-statistic, we can use the $F$-distribution with $N-K$ and $K-1$ degrees of degrees of freedom to get a p-value.<br>
<br>
**Warning!** The $F$-statistic mentioned in this section is NOT the same as the F1-measure or F1-value discused in Unit 7.

<div class="span5 alert alert-info"><br>
<h3>Part 3 Checkup Exercise Set IV</h3><br>
<br>
<p>Let's look at the relationship between `PTRATIO` and housing price.</p><br>
<br>
<p><b>Exercise:</b> Try fitting a linear regression model using only the 'PTRATIO' (pupil-teacher ratio by town) and interpret the intercept and the coefficients.</p><br>
<br>
<p><b>Exercise:</b> Calculate (or extract) the $R^2$ value. What does it tell you?</p><br>
<br>
<p><b>Exercise:</b> Compute the $F$-statistic. What does it tell you?</p><br>
<br>
<p><b>Exercise:</b> Take a close look at the $F$-statistic and the $t$-statistic for the regression coefficient. What relationship do you notice? Note that this relationship only applies in *simple* linear regression models.</p><br>
</div>

In[44]:

your turn

In [ ]:
n = ols('PTRATIO ~ RM',bos).fit()
print(n.summary())

In[61]:

In [ ]:
lm.fit(bos.PTRATIO.values.reshape(-1,1) , bos.PRICE)
print("Coef: ", lm.coef_)
print("Intercept: ", lm.intercept_)
rsq=(lm.score(bos.PTRATIO.values.reshape(-1, 1), bos.PRICE))
print("R square: {}".format(rsq))
radj= 1-(1-rsq)*((len(bos.PTRATIO)-1)/(len(bos.PTRATIO)-1-1))
print ("R adj: ",radj)
fstat= (rsq/(1-rsq))*((len(bos.PTRATIO)-1-1)/1)
print("F statistic: ",fstat)
p = 1-stats.f.cdf(fstat, len(bos.PTRATIO), len(bos.PTRATIO))
print("P value: ",p)

Only 25% of PTRATIO values can accurately predict the prices with a very high value of F statistic and P value less than 0.05 which does show statistical significance 

<div class="span5 alert alert-info"><br>
<h3>Part 3 Checkup Exercise Set V</h3><br>
<br>
<p>Fit a linear regression model using three independent variables</p><br>
<br>
<ol><br>
<li> 'CRIM' (per capita crime rate by town)<br>
<li> 'RM' (average number of rooms per dwelling)<br>
<li> 'PTRATIO' (pupil-teacher ratio by town)<br>
</ol><br>
<br>
<p><b>Exercise:</b> Compute or extract the $F$-statistic. What does it tell you about the model?</p><br>
<br>
<p><b>Exercise:</b> Compute or extract the $R^2$ statistic. What does it tell you about the model?</p><br>
<br>
<p><b>Exercise:</b> Which variables in the model are significant in predicting house price? Write a story that interprets the coefficients.</p><br>
</div>

In[62]:

your turn

In [ ]:
lm.fit(bos[["CRIM","RM","PTRATIO"]], bos.PRICE)
rsq=lm.score(bos[["CRIM","RM","PTRATIO"]], bos.PRICE)
print("Coefficient Print\nCRIM: %f   RM: %f   PTRATIO: %f" % (lm.coef_[0], lm.coef_[1], lm.coef_[2]))
print("R square: {}".format(rsq))
radj= 1-(1-rsq)*((len(bos.PTRATIO)-1)/(len(bos.PTRATIO)-3-1))
print ("R adj: ",radj)
fstat= (rsq/(1-rsq))*((len(bos.PTRATIO)-3-1)/3)
print("F statistic: ",fstat)
p = 1-stats.f.cdf(fstat, len(bos.PTRATIO), len(bos.PTRATIO))
print("P value: ",p)

59 percent of the predictions can be explained by +7.38 times PTRATIO, -1.06 times RM and -0.20 times CRIM with a p value less than 0.05 making it statistically significant.

## Part 4: Comparing Models

During modeling, there will be times when we want to compare models to see which one is more predictive or fits the data better. There are many ways to compare models, but we will focus on two.

### The $F$-Statistic Revisited<br>
<br>
The $F$-statistic can also be used to compare two *nested* models, that is, two models trained on the same dataset where one of the models contains a *subset* of the variables of the other model. The *full* model contains $K$ variables and the *reduced* model contains a subset of these $K$ variables. This allows us to add additional variables to a base model and then test if adding the variables helped the model fit.<br>
<br>
$$F = \frac{\left( \frac{RSS_{reduced} - RSS_{full}}{DF_{reduced} - DF_{full}} \right)}{\left( \frac{RSS_{full}}{DF_{full}} \right)}$$<br>
<br>
where $DF_x = N - K_x - 1$ where $K_x$ is the number of variables in model $x$.

### Akaike Information Criterion (AIC)<br>
<br>
Another statistic for comparing two models is AIC, which is based on the likelihood function and takes into account the number of variables in the model.<br>
<br>
$$AIC = 2 K - 2 \log_e{L}$$<br>
<br>
where $L$ is the likelihood of the model. AIC is meaningless in the absolute sense, and is only meaningful when compared to AIC values from other models. Lower values of AIC indicate better fitting models.<br>
<br>
`statsmodels` provides the AIC in its output.

<div class="span5 alert alert-info"><br>
<h3>Part 4 Checkup Exercises</h3><br>
<br>
<p><b>Exercise:</b> Find another variable (or two) to add to the model we built in Part 3. Compute the $F$-test comparing the two models as well as the AIC. Which model is better?</p><br>
</div>

In[63]:

In [ ]:
m1 = ols('PRICE ~ CRIM + RM + PTRATIO',bos).fit()
print(m1.summary())
m2 = ols('PRICE ~ CRIM + RM + PTRATIO + AGE + DIS',bos).fit()
print(m2.summary())

R square or adjusted R square value seems to be higher for model 2, however this could also be due to the increase in independent variables and not accuracy. Model 2 has a lesser AIC value suggesting higher performance than model 1 with both having statistical significance, with Model 1 having higher F statistic value means higher variance.

<br>
## Part 5: Evaluating the Model via Model Assumptions and Other Issues<br>
***<br>
Linear regression makes several assumptions. It is always best to check that these assumptions are valid after fitting a linear regression model.<br>
<br>
<div class="span5 alert alert-danger"><br>
<ul><br>
  <li>**Linearity**. The dependent variable $Y$ is a linear combination of the regression coefficients and the independent variables $X$. This can be verified with a scatterplot of each $X$ vs. $Y$ and plotting correlations among $X$. Nonlinearity can sometimes be resolved by [transforming](https://onlinecourses.science.psu.edu/stat501/node/318) one or more independent variables, the dependent variable, or both. In other cases, a [generalized linear model](https://en.wikipedia.org/wiki/Generalized_linear_model) or a [nonlinear model](https://en.wikipedia.org/wiki/Nonlinear_regression) may be warranted.</li><br>
  <li>**Constant standard deviation**. The SD of the dependent variable $Y$ should be constant for different values of X. We can check this by plotting each $X$ against $Y$ and verifying that there is no "funnel" shape showing data points fanning out as $X$ increases or decreases. Some techniques for dealing with non-constant variance include weighted least squares (WLS), [robust standard errors](https://en.wikipedia.org/wiki/Heteroscedasticity-consistent_standard_errors), or variance stabilizing transformations.<br>
    </li><br>
  <li> **Normal distribution for errors**.  The $\epsilon$ term we discussed at the beginning are assumed to be normally distributed. This can be verified with a fitted values vs. residuals plot and verifying that there is no pattern, and with a quantile plot.<br>
  $$ \epsilon_i \sim N(0, \sigma^2)$$<br>
Sometimes the distributions of responses $Y$ may not be normally distributed at any given value of $X$.  e.g. skewed positively or negatively. </li><br>
<li> **Independent errors**.  The observations are assumed to be obtained independently.<br>
    <ul><br>
        <li>e.g. Observations across time may be correlated<br>
    </ul><br>
</li><br>
</ul>  <br>
<br>
</div><br>
<br>
There are some other issues that are important investigate with linear regression models.<br>
<br>
<div class="span5 alert alert-danger"><br>
<ul><br>
  <li>**Correlated Predictors:** Care should be taken to make sure that the independent variables in a regression model are not too highly correlated. Correlated predictors typically do not majorly affect prediction, but do inflate standard errors of coefficients making interpretation unreliable. Common solutions are dropping the least important variables involved in the correlations, using regularlization, or, when many predictors are highly correlated, considering a dimension reduction technique such as principal component analysis (PCA).<br>
  <li>**Influential Points:** Data points that have undue influence on the regression model. These points can be high leverage points or outliers. Such points are typically removed and the regression model rerun.<br>
</ul><br>
</div><br>


<div class="span5 alert alert-info"><br>
<h3>Part 5 Checkup Exercises</h3><br>
<br>
<p>Take the reduced model from Part 3 to answer the following exercises. Take a look at [this blog post](http://mpastell.com/2013/04/19/python_regression/) for more information on using statsmodels to construct these plots.</p><br>
    <br>
<p><b>Exercise:</b> Construct a fitted values versus residuals plot. What does the plot tell you? Are there any violations of the model assumptions?</p><br>
<br>
<p><b>Exercise:</b> Construct a quantile plot of the residuals. What does the plot tell you?</p><br>
<br>
<p><b>Exercise:</b> What are some advantages and disadvantages of the fitted vs. residual and quantile plot compared to each other?</p><br>
<br>
<p><b>Exercise:</b> Identify any outliers (if any) in your model and write a story describing what these outliers might represent.</p><br>
<br>
<p><b>Exercise:</b> Construct a leverage plot and identify high leverage points in the model. Write a story explaining possible reasons for the high leverage points.</p><br>
<br>
<p><b>Exercise:</b> Remove the outliers and high leverage points from your model and run the regression again. How do the results change?</p><br>
</div>

In[67]:

Your turn.

In [ ]:
plt.scatter(lm.predict(bos[["CRIM","RM","PTRATIO"]]), bos.PRICE - lm.predict(bos[["CRIM","RM","PTRATIO"]]))
plt.xlabel("Predicted Price")
plt.ylabel("Residual")
plt.title("Residual Plot")
plt.show()

In [ ]:
plt.hist(bos.PRICE - lm.predict(bos[["CRIM","RM","PTRATIO"]]),bins = 30)
plt.xlabel("Residual")
plt.ylabel("Count")
plt.title("Residual Histogram")
plt.show()

There is clearly a relationship between residuals and the predicted values , When the residuals center on zero they indicate that the modelâ€™s predictions are correct on average rather than systematically too high or low. Regression also assumes that the residuals follow a normal distribution and that the degree of scattering is the same for all fitted values. There seems to be outliers for Price greater than 20

In[68]:

In [ ]:
stats.probplot(bos.PRICE - lm.predict(bos[["CRIM","RM","PTRATIO"]]), fit=True, plot=plt)
plt.show()
# residuals showing normality in most cases except for prices above 20 as noted earlier

In[69]:

In [ ]:
import statsmodels
plt.rcParams["figure.figsize"] = (20,20)
statsmodels.graphics.regressionplots.influence_plot(m1)
plt.show()

We have high leverage points in the extreme right of the x axis which means those rows which differ significantly from other independent rows<br>
and ones with high residual values after 4.Lets evaluate the performance by removing them.

In[70]:

In [ ]:
bos1 =bos.drop(bos.index[[414,410,405,418,380,367,370,369,365,371,372,368]]).reindex()

In[72]:

your turn

In [ ]:
lm.fit(bos1[["CRIM","RM","PTRATIO"]], bos1.PRICE)
rsq=lm.score(bos1[["CRIM","RM","PTRATIO"]], bos1.PRICE)
print("Coefficient Print\nCRIM: %f   RM: %f   PTRATIO: %f" % (lm.coef_[0], lm.coef_[1], lm.coef_[2]))
print("R square: {}".format(rsq))
radj= 1-(1-rsq)*((len(bos1.PTRATIO)-1)/(len(bos1.PTRATIO)-3-1))
print ("R adj: ",radj)
fstat= (rsq/(1-rsq))*((len(bos1.PTRATIO)-3-1)/3)
print("F statistic: ",fstat)
p = 1-stats.f.cdf(fstat, len(bos1.PTRATIO), len(bos1.PTRATIO))
print("P value: ",p)
plt.scatter(bos1.PRICE, lm.predict(bos1[["CRIM","RM","PTRATIO"]]))
plt.xlabel("Observed Price")
plt.ylabel("Predicted Price")
plt.title("Observed Price vs Predicted Price")

In[73]:

In [ ]:
m1 = ols('PRICE ~ CRIM + RM + PTRATIO',bos).fit()
print(m1.summary())
m3 = ols('PRICE ~ CRIM + RM + PTRATIO',bos1).fit()
print(m3.summary())

R square /adjusted r square value has increased from 0.59 to .72 after outlier removal suggesting further increase in dependent and independent variable relationships. Values seems to be statistically significant as well with a p value < 0.05 and lesser AIC value.<br>
<br>
Although it does not mean outlier removed model best represents the data, its just the best fit linear model.